# Ejemplo 02

En este ejemplo vamos a tomar dos individuos y realizar la cruza de ellos. La cruza siempre arrojará 2 individuos nuevos, con los genes intercambiados. Esto hace que los individuos mezclen sus genes para evitar que se estanquen.

Primero, vamos a tomar las funciones del ejemplo 01. 

In [1]:
import numpy as np
import random as rnd

def inicializarIndividuo(individuo):
    total = 1.0
    cantidadGenes = len(individuo)
    for counter in range(0,cantidadGenes):
        if(counter == cantidadGenes -1):
            individuo[counter] = total
        else:  
            randomValue = rnd.uniform(0,total)
            individuo[counter] = randomValue
            total -=randomValue
    return individuo

def generarPoblacion(cantidadDeIndividuos, cantidadDeGenes):
    poblacion = np.zeros((cantidadDeIndividuos, cantidadDeGenes))
    for counter in range(0,cantidadDeIndividuos):
        poblacion[counter] = inicializarIndividuo(poblacion[counter])
    return poblacion

### Ajuste de genes.
Un paso importante antes de continuar es el ajuste de los genes de un individuo:

De nada nos sirve un indviduo que se pase del 100% al sumar sus genes, o que no llegue al 100%. 
Por ende, esta función "normaliza" al individuo, de tal manera que la suma de sus genes siempre sea 1.0, sin importar si hubo cambios. 

Esto puede variar dependiendo de la aplicación que tengas, pero para porcentajes, es idóneo.

In [2]:
def ajustarIndividuo(individuo):
    total = np.sum(individuo)
    individuo = individuo / total
    return individuo    

## Cruza de individuos
Dependiendo de un punto de cruza aleatorio (que puede ser entre cualquiera de los genes), se generan dos individuos hijo: uno con ciertos genes del padre y el resto de la madre. El segundo hijo es inverso, de tal manera que si el padre tiene un genotipo:

- p p p p 

y la madre:
- m m m m

Entonces si asumimos que el punto de cruza es entre el gen 1 y 2, el hijo 1 es:

- p p m m

y el hijo 2 es:

- m m p p

Por evidentes razones, cuando hacemos la cruza, se deben ajustar los nuevos individuos, ya que al cruzar no podemos asegurar que los genes sumen 1.0

In [3]:
def cruzarIndividuos(individuo1, individuo2):
    crossPoint = rnd.randint(1,len(individuo1)-2)
    hijo1 = np.append(individuo1[0:crossPoint], individuo2[crossPoint:len(individuo1)])
    hijo2 = np.append(individuo2[0:crossPoint], individuo1[crossPoint:len(individuo1)])
    hijo1 = ajustarIndividuo(hijo1)
    hijo2 = ajustarIndividuo(hijo2)
    return np.array([hijo1,hijo2])

## Cruza poblacional.
Aún cuando un individo sea idóneo y apto, no necesariamente quiere decir que se cruzará. Hay una probabilidad de cruza, y esta se lleva a cabo de la siguiente manera: 
- Se determina aleatoriamente si un individuo se cruzará. si no se cruza, no pasa nada.
- Si se cruza, se debe elegir a algun otro individuo al azar.
- Se realiza la operación de cruza entre dos individuos y los hijos se agregan a una lista nueva de población.
- Finalmente, la lista de nuevos individuos se agregan a la población. 

In [4]:
def cruzarPoblacion(poblacion, probabilidadDeCruza):    
    nuevaPoblacion = None
    for counter in range(len(poblacion)):
        probabilidad = rnd.random()
        #Si la probabilidad es menor que la probabilidad de cruza, entonces se procede a la cruza.
        if(probabilidad < probabilidadDeCruza):
            #Elige pareja:
            pareja = rnd.randint(0,len(poblacion)-2)
            #Si al elegir una pareja, elegiste sin querer al mismo individuo, intenta de nuevo.
            while pareja == counter:
                pareja = rnd.randint(0,len(poblacion)-2)
            
            #Haz la cruza. 
            nuevosIndividuos = cruzarIndividuos(poblacion[counter],poblacion[pareja])
            
            #Añade los nuevos individuos a una lista de nuevos individuos.
            if nuevaPoblacion is None:
                nuevaPoblacion = nuevosIndividuos
            else:
                nuevaPoblacion = np.concatenate((nuevaPoblacion,nuevosIndividuos))
                
    #Al final del ciclo, si hay nuevos individuos, añadelos a la población.
    if not nuevaPoblacion is None:
        poblacion = np.concatenate((poblacion, nuevaPoblacion))
    return poblacion

## Probar la cruza.
Hagamos una cruza de datos y veamos que tal se ve!

In [6]:
#6 individuos, 4 genes cada uno
poblacion = generarPoblacion(6,4)
print("Poblacion original:")
print(poblacion)

#50% de probabilidad de cruza:
poblacion = cruzarPoblacion(poblacion, 0.5)
print("Población con hijos añadidos:")
print(poblacion)

Poblacion original:
[[4.91618022e-01 4.50119808e-01 1.63786448e-02 4.18835250e-02]
 [2.77237130e-01 1.06281232e-01 2.35366946e-01 3.81114691e-01]
 [2.04291166e-01 6.80675739e-01 3.38676982e-02 8.11653975e-02]
 [6.40668934e-01 3.01733276e-02 1.12247888e-01 2.16909850e-01]
 [9.94807566e-01 4.30742326e-03 3.75716772e-04 5.09294157e-04]
 [7.24093581e-01 1.10531297e-01 2.81906889e-02 1.37184433e-01]]
Población con hijos añadidos:
[[4.91618022e-01 4.50119808e-01 1.63786448e-02 4.18835250e-02]
 [2.77237130e-01 1.06281232e-01 2.35366946e-01 3.81114691e-01]
 [2.04291166e-01 6.80675739e-01 3.38676982e-02 8.11653975e-02]
 [6.40668934e-01 3.01733276e-02 1.12247888e-01 2.16909850e-01]
 [9.94807566e-01 4.30742326e-03 3.75716772e-04 5.09294157e-04]
 [7.24093581e-01 1.10531297e-01 2.81906889e-02 1.37184433e-01]
 [4.65207747e-01 4.25938865e-01 3.20482873e-02 7.68051008e-02]
 [2.16587011e-01 7.21644145e-01 1.73644401e-02 4.44044041e-02]
 [5.56085287e-01 2.13180065e-01 6.79322017e-02 1.62802447e-01]
 [1.